<a href="https://colab.research.google.com/github/ashikshafi08/Recipe1M/blob/main/untidy/Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import json
import os 

det_path = '/content/drive/MyDrive/Recipe_1M/det_ingrs.json' 
layer1_path = '/content/drive/MyDrive/Recipe_1M/layer1.json'
layer2_path = '/content/drive/MyDrive/Recipe_1M/layer2.json'


In [2]:

# Loading the layer 1 json file 
with open(layer1_path , 'r') as file:
  layer1 = json.load(file)

In [3]:

# Loading the layer 2 json file 
with open(layer2_path , 'r') as file:
  layer2 = json.load(file)

In [4]:

# Loading the detr_ingrs json file 
with open(det_path , 'r') as file:
  dets = json.load(file)

In [5]:
# Parsing the id and storing it as indices for further indexing!
idx2ind = {} # empty dictionary

for i , entry in enumerate(dets):
  idx2ind[entry['id']] = i 

len(idx2ind)

1029720

In [34]:
replace_dict = {' .': '.',
                ' ,': ',',
                ' ;': ';',
                ' :': ':',
                '( ': '(',
                ' )': ')',
               " '": "'"}

Now we got the `det_ingrs` which has the ingredients text stored in a dictionary format, lets create a function to parse out the ingredients in a more human readable way. 

But before writing the function lemme break each and every single line 


In [48]:
def get_ingredient(det_ingr , replace_dict):
  '''
  det_ingr -> single dictionary text with ingredients as values 
  replace_dict -> a dictionary for replacing the texts 

  Here we pass a dictionary of ingredients text 
  '''
  # Lowering all the ingredients text
  det_ingr_undrs = det_ingr['text'].lower()
  det_ingr_undrs = ''.join(i for i in det_ingr_undrs if not i.isdigit())

  # Looping through the replace dictionary which has special characters in it 
  for rep , char_list in replace_dict.items():
    for c_ in char_list:
      if c_ in det_ingr_undrs:
        det_ingr_undrs = det_ingr_undrs.replace(c_ , rep) # Replacing some of the characters into the values in dictionary

  det_ingr_undrs = det_ingr_undrs.strip() 
  det_ingr_undrs = det_ingr_undrs.replace(' ' , '_')

  return det_ingr_undrs


In [56]:
from tqdm import tqdm 

for i , entry in tqdm(enumerate(layer1)):

  # get all instructions ]
  instrs = entry['instructions']

  # Instruction and ingredients list 
  instrs_list = [] 
  ingrs_list = []

  # Retrieve the ingredients for this entry 
  det_ingrs = dets[idx2ind[entry['id']]]['ingredients']

  # Getting the valid values [True]
  valid = dets[idx2ind[entry['id']]]['valid']
  det_ingrs_filtered  = []

  # Now applying the function we've created above 
  for j , det_ingr in enumerate(det_ingrs):
    if len(det_ingr) > 0 and valid[j]:
      det_ingr_undrs = get_ingredient(det_ingr , replace_dict_ingrs)
      det_ingrs_filtered.append(det_ingr_undrs)
      ingrs_list.append(det_ingr_undrs)
  


len(ingrs_list)

1029720it [00:52, 19629.60it/s]


5

In [57]:
ingrs_list

['water', 'light_brown_sugar', 'molasses', 'cinnamon_sticks', 'coffee']

In [50]:
replace_dict_ingrs = {'and': ['&', "'n"], '': ['%', ',', '.', '#', '[', ']', '!', '?']}
# The above sample but with just one dictionary 
for j, det_ingr in enumerate(det_ingrs):
  if len(det_ingr) > 0 and valid[j]:
    #print(det_ingr)
    sample_undrs = det_ingr['text'].lower() # Lower case 
    ''.join(i for i in sample_undrs if not i.isdigit()) # Checking if there is any digit in the string by looping through a string 

  for rep , char_list in replace_dict_ingrs.items():
    for c_ in char_list:
      for c_ in sample_undrs:
        new_sample_undrs = sample_undrs.replace(c_ , rep)
  sample_det_ingr_undrs = new_sample_undrs.strip()
  print(sample_det_ingr_undrs.replace(' ' , '_'))
    


flou
ciamo
bking_sod
sal
baking_powde
egg
suga
vegetabe_oi
vnill
raw_zucchn
walnut


Rewriting the `build_vocab_recipe1m` function before jumping into the code lets write in simple english what they are trying to do: 

1. First we load the data from our json file 
2.  Looping through the `layer2` json file and grabbing the corresponding image id that matches the unique id. We will name it as `id2im`
3. Then we print out saying how many recipes are there in the dataset. `len(layer1)`
4. We assign a dictionary for both the ingredients and instruction that assigns some special character for certain keys. 
5. Parsing the id and storing it as indices for further indexing!

Here comes our first loop where we iterate into the `layer1` json file that has everything from the unique id to the instructions. 

**Iterating into the `layer1`**
- Storing all the instructions into a variable `instrs`, 
- Creating two empty list saying instruction and ingredients list. 
- Storing the ingredients by indexing [`idx2ind`] into the `det_ingrs.json` file. 
  - Also we filter out the validation files from the `det_ingrs.json` and store them in a `valid` variable that contains boolean values. 
  - **Iterating into the `det_ingrs` that has ingredients**
    - We use enumerate to get even the counts and iterate into the ingredients dictionary. 
    - Putting up a condition saying if the length of the ingredient should be greater than 0 and also `valid` should be True. Then proceed.....
    - Applying the `get_ingredient` function on each of our ingredient dictionary with a key ['text'] in it. (Will write about this function later...) [**`get_ingredient()`**]
    - Then we append the filtered out ingredients into a list called `det_ingrs_filtered` and `ingrs_list` for future use. 
 - **Iterating into the instructions dictionary**
    - Creating a empty variable called `acc_len`. 
    - Iterating into the `instrs` dictionary with an variable `instr`. 
    - Assigning the instruction 'text' to the variable `instr`
    - Applying a function `get_instruction()` that will do the same what `get_ingredient` does. 






In [58]:
layer2[0]

{'id': '00003a70b1',
 'images': [{'id': '3e233001e2.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picaYYmb9.jpg'},
  {'id': '7f749987f9.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picpy37SW.jpg'},
  {'id': 'aaf6b2dcd3.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picX9CNE2.jpg'}]}